<a href="https://colab.research.google.com/github/aelydens/aie-resources/blob/main/AL_RAG_NVIDIA_10k_Filings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NVIDIA 10k Filings RAG with Langchain

Deliverables:

Build 🏗️
* Data: NVIDIA 10-k Filings
* Model: OpenAI text-3-embedding small, GPT-3.5-turbo
* Tooling: LangChain or LlamaIndex (you choose)
* Vector Store: FAISS
* Additional Component: Add one of the following: 1) visibility with WandB OR 2) evaluation with RAGAS
Ship 🚢

Evaluate your answers to the following questions
* "Who is the E-VP, Operations - and how old are they?"
* "What is the gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023?"

In [1]:
#  installing the required packages and -qU for quiet and upgrade installation process
!pip install -qU langchain langchain-core langchain-community langchain-openai pymupdf faiss_cpu

In [17]:
# importing required modules
import os
import getpass
import wandb

In [3]:
# This line of code allows Jupyter Notebook to access the OpenAI API key securely without displaying it in the code.
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [18]:
# This line of code is setting the value of the environment variable "WANDB_API_KEY" to the result of the getpass.getpass() function.
os.environ["WANDB_API_KEY"] = getpass.getpass("WandB API Key: ")

In [5]:
# This line of code is setting an environment variable named "LANGCHAIN_WANDB_TRACING" to the value "true".
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"

In [6]:
# Initializing the OpenAI Embedding Model `text-embedding-3-small`.
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [7]:
# Loading and spliting the document.
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyMuPDFLoader("https://d18rn0p25nwr6d.cloudfront.net/CIK-0001045810/1cbe8fe7-e08a-46e3-8dcc-b429fc06c1a4.pdf")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

In [8]:
# Setting up FAISS-powered vector store, and creating a retriever.
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, embeddings)

retriever = vector_store.as_retriever()

In [9]:
# Creating prompt using `ChatPromptTemplate`.
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [10]:
# Initializing OpenAI Chat Model 'gpt-3.5-turbo' with a temperature of 0.
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [11]:
# Creating LCEL chain.
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | openai_chat_model, "context": itemgetter("context")}
)

In [12]:
# Importing the required modules, Creating a Retrieval Augmented QA Chain and initializing Weights & Biases project.
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

retrieval_augmented_qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(temperature=0.3),
    chain_type="stuff",
    retriever=retriever,
)

wandb.init(project="Midterm-Certification")

/home/sysadmin/anaconda3/envs/llmops-course/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: waqasjavedkhan. Use `wandb login --relogin` to force relogin


In [19]:
# Running evaluation and logging the results.
print(retrieval_augmented_qa_chain.run("Who is the E-VP, Operations - and how old are they?"))

wandb: Streaming LangChain activity to W&B at https://wandb.ai/waqasjavedkhan/AIE-Week%204%20-%20Midterm%20Certification/runs/1mz625m2
wandb: `WandbTracer` is currently in beta.
wandb: Please report any issues to https://github.com/wandb/wandb/issues with the tag `langchain`.
wandb: WARNING WARNING: Failed to serialize model: 'int' object is not subscriptable


 Debora Shoquist is the Executive Vice President of Operations at NVIDIA and is 69 years old.


In [14]:
# Running evaluation and logging the results.
print(retrieval_augmented_qa_chain.run("What is the gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023 in billions?"))

wandb: WARNING WARNING: Failed to serialize model: 'int' object is not subscriptable


 The gross carrying amount of Total Amortizable Intangible Assets for Jan 29, 2023 is $3.539 billion.


In [16]:
# Closing the logging.
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
